In [6]:
import { z } from "npm:zod";
const stringSchema = z.string();
stringSchema.parse("Hello, Zod!");

"Hello, Zod!"

In [10]:
stringSchema.parse(2323);

ZodError: [
  {
    "code": "invalid_type",
    "expected": "string",
    "received": "number",
    "path": [],
    "message": "Expected string, received number"
  }
]

In [11]:
// 基础类型
const stringSchema = z.string();
const numberSchema = z.number();
const booleanSchema = z.boolean();

// 数组
const stringArraySchema = z.array(z.string());
stringArraySchema.parse(["apple", "banana", "cherry"]); 

// 对象
const personSchema = z.object({
  name: z.string(),
  age: z.number(),
  // 可选类型
  isStudent: z.boolean().optional(),
  // 默认值
  home: z.string().default("no home")
});

// 联合类型
const mixedTypeSchema = z.union([z.string(), z.number()]);
mixedTypeSchema.parse("hello"); 
mixedTypeSchema.parse(42); 


42

In [13]:
import {zodToJsonSchema }from 'zod-to-json-schema'
const getCurrentWeatherSchema = z.object({
  location: z.string().describe("The city and state, e.g. San Francisco, CA"),
  unit: z.enum(["celsius", "fahrenheit"]).describe("The unit of temperature"),
});
const paramSchema = zodToJsonSchema(getCurrentWeatherSchema)
paramSchema

{
  type: "object",
  properties: {
    location: {
      type: "string",
      description: "The city and state, e.g. San Francisco, CA"
    },
    unit: {
      type: "string",
      enum: [ "celsius", "fahrenheit" ],
      description: "The unit of temperature"
    }
  },
  required: [ "location", "unit" ],
  additionalProperties: false,
  "$schema": "http://json-schema.org/draft-07/schema#"
}

In [16]:
import {ChatDeepSeek} from '@langchain/deepseek'
import {zodToJsonSchema }from 'zod-to-json-schema'
import { ChatPromptTemplate } from "@langchain/core/prompts";
const model = new ChatDeepSeek({
    model:'deepseek-chat',
    temperature:0,
})
const modelWithTools = model.bind({
    tools: [
        {
            type: "function",
            function: {
                name: "getCurrentWeather",
                description: "Get the current weather in a given location",
                parameters: zodToJsonSchema(getCurrentWeatherSchema),
            }
        }
    ]
})
const prompt = ChatPromptTemplate.fromMessages([
    ["system", "You are a helpful assistant"],
    ["human", "{input}"]
])
const chain = prompt.pipe(modelWithTools)
await chain.invoke({
    input: "北京的天气怎么样"
});

AIMessage {
  "id": "e0fb305f-4efb-47d8-abfd-8557dcb01a9f",
  "content": "",
  "additional_kwargs": {
    "tool_calls": [
      {
        "index": 0,
        "id": "call_0_9a23b7c6-8023-43d5-a599-133a8abec438",
        "type": "function",
        "function": "[Object]"
      }
    ]
  },
  "response_metadata": {
    "tokenUsage": {
      "promptTokens": 196,
      "completionTokens": 24,
      "totalTokens": 220
    },
    "finish_reason": "tool_calls",
    "model_name": "deepseek-chat",
    "usage": {
      "prompt_tokens": 196,
      "completion_tokens": 24,
      "total_tokens": 220,
      "prompt_tokens_details": {
        "cached_tokens": 0
      },
      "prompt_cache_hit_tokens": 0,
      "prompt_cache_miss_tokens": 196
    },
    "system_fingerprint": "fp_3a5770e1b4_prod0225"
  },
  "tool_calls": [
    {
      "name": "getCurrentWeather",
      "args": {
        "location": "北京",
        "unit": "celsius"
      },
      "type": "tool_call",
      "id": "call_0_9a23b7c6-8023-43d

In [ ]:
const getCurrentTimeSchema = z.object({
  format: z
    .enum(["iso", "locale", "string"])
    .optional()
    .describe("The format of the time, e.g. iso, locale, string"),
});

const modelWithMultiTools = model.bind({
    tools: [
        {
            type: "function",
            function: {
                name: "getCurrentWeather",
                description: "Get the current weather in a given location",
                parameters: zodToJsonSchema(getCurrentWeatherSchema)
            }
        },
        {
            type: "function",
            function: {
                name: "getCurrentTime",
                description: "Get the current time in a given format",
                parameters: zodToJsonSchema(getCurrentTimeSchema)
            }
        }
    ]
})
